### 🧭 Summary (Вступ)

Markdown — це проста й потужна мова розмітки, що дозволяє красиво документувати код, створювати пояснення, таблиці, формули та навіть елементи дизайну без необхідності знати HTML.
У середовищі **Jupyter Notebook** Markdown використовується для створення текстових блоків між комірками коду, що робить аналітичні звіти, дослідження даних і навчальні матеріали читабельними й професійними.

Markdown допомагає структурувати роботу, пояснювати алгоритми, вставляти зображення, посилання, формули LaTeX і навіть форматовані повідомлення (alert boxes).


<div class="alert alert-success">
<b>Success:</b> Data loaded successfully!
</div>

https://www.kaggle.com/datasets/yasserh/titanic-dataset


In [2]:
!pip install kagglehub

100%|██████████| 22.0k/22.0k [00:00<00:00, 31.9MB/s]

Extracting files...
Шлях до датасету: /home/repl/.cache/kagglehub/datasets/yasserh/titanic-dataset/versions/1


=== Перші 5 рядків датасету ===
   PassengerId  Survived  Pclass  ...     Fare Cabin  Embarked
0            1         0       3  ...   7.2500   NaN         S
1            2         1       1  ...  71.2833   C85         C
2            3         1       3  ...   7.9250   NaN         S
3            4         1       1  ...  53.1000  C123         S
4            5         0       3  ...   8.0500   NaN         S

[5 rows x 12 columns]


=== Розмір датасету ===
Кількість рядків: 891
Кількість колонок: 12


=== Назви колонок ===
['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']


=== Інформація про датасет ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 

In [3]:
import pandas as pd

# Завантажуємо датасет Titanic напряму з URL
url = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'
df = pd.read_csv(url)

# 1. Подивимось на перші 5 рядків
print("=== Перші 5 рядків датасету ===")
print(df.head())
print("\n")

# 2. Розмір датасету (рядки, колонки)
print(f"=== Розмір датасету ===")
print(f"Кількість рядків: {df.shape[0]}")
print(f"Кількість колонок: {df.shape[1]}")
print("\n")

# 3. Назви всіх колонок
print("=== Назви колонок ===")
print(df.columns.tolist())
print("\n")

# 4. Базова інформація про датасет
print("=== Інформація про датасет ===")
print(df.info())
print("\n")

# 5. Статистика по числових колонках
print("=== Статистика ===")
print(df.describe())
print("\n")

# 6. Скільки людей вижило?
print("=== Виживання пасажирів ===")
print(f"Вижило: {df['Survived'].sum()} людей")
print(f"Загинуло: {len(df) - df['Survived'].sum()} людей")
print("\n")

# 7. Середній вік пасажирів
print("=== Вік пасажирів ===")
print(f"Середній вік: {df['Age'].mean():.1f} років")
print("\n")

# 8. Кількість чоловіків і жінок
print("=== Розподіл за статтю ===")
print(df['Sex'].value_counts())
print("\n")

# 9. Виживання по класах
print("=== Виживання по класах ===")
survival_by_class = df.groupby('Pclass')['Survived'].mean() * 100
print(survival_by_class)
print("\n")

# 10. Збереження обробленого датасету
# df.to_csv('titanic_processed.csv', index=False)
# print("Датасет збережено у файл titanic_processed.csv")

=== Перші 5 рядків датасету ===
   PassengerId  Survived  Pclass  ...     Fare Cabin  Embarked
0            1         0       3  ...   7.2500   NaN         S
1            2         1       1  ...  71.2833   C85         C
2            3         1       3  ...   7.9250   NaN         S
3            4         1       1  ...  53.1000  C123         S
4            5         0       3  ...   8.0500   NaN         S

[5 rows x 12 columns]


=== Розмір датасету ===
Кількість рядків: 891
Кількість колонок: 12


=== Назви колонок ===
['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']


=== Інформація про датасет ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Nam